# Deploy Logic App + ACI

In this notebook, we will perform the final deployment so that the whole architecture is on Azure.

### Recap of the architecture:
The architecture for this workflow is centered on two main components:
- The AKS cluster that will perform the scoring by pulling (references of) images off the queue
- The Logic App + ACI that will trigger the workflow and populate the queue

At this point, we already have an AKS cluster up and running. It is continuously polling out Service Bus queue to check if there are any incoming messages to process. 

The next step is to deploy our Logic App + ACI components so that the workflow can be triggered by new videos that appear in blob storage. Once a new video is found in blob, Logic App will trigger the creation of an ACI container that will run the video preprocessing script, the add-to-queue script, and finally the postprocessing script. 

As soon as new items are added to the queue (from ACI), our AKS cluster will start pulling those items off and start processing them.

### Steps in this notebook:
1. Create the Docker image that will run inside of our ACI container
2. Test the Docker image by runnning it locally. 
3. Deploy the Logic App. The Logic App will go live once deployed. 
4. Test the Logic App deployment. To do so, we upload a new video to blob, which will triger the entire workflow.

---

### Import packages and load .env

In [7]:
from dotenv import set_key, get_key, find_dotenv, load_dotenv
from pathlib import Path
import json
import jinja2
import os

In [8]:
env_path = find_dotenv(raise_error_if_not_found=True)
load_dotenv(env_path)

True

### Deploy Logic App

![Logic Apps](https://happypathspublic.blob.core.windows.net/assets/batch_scoring_for_dl/azure_logic_app.PNG)

The *logic* behind the Logic App deployment is shown above:
1. When a blob is added, begin the workflow.
2. Check the blob name. 
    - if the blob name ends with `.mp4`:
        - create an ACI 
    - otherwise:
        - terminate in cancellation


Using Jinja, populate the `template.logic_app.json` file and output the new file as `logic_app.json`. This file will be saved in the working directory.

In [ ]:
# use jinja to fill in variables from .env file
env = jinja2.Environment(
    loader=jinja2.FileSystemLoader('.')
)
template = env.get_template('template.logic_app.json')

e = os.environ
rendered_template = template.render(env=e)

out = open('logic_app.json', 'w')
out.write(rendered_template)
out.close()

Deploy our ARM template `logic_app.json`.

In [ ]:
!az group deployment create \
    --name {get_key(env_path, "LOGIC_APP")} \
    --resource-group {get_key(env_path, "RESOURCE_GROUP")} \
    --template-file logic_app.json

Once the Logic App is deployed, go into the Azure portal and open up the ACI connector and the Azure blob connector to authenticate. 

When you open up up the Azure ACI connector, it should look like this:

![azure_aci_connector_auth](https://happypathspublic.blob.core.windows.net/assets/batch_scoring_for_dl/azure_aci_connector_auth.PNG)

When you open up up the Azure blob connector, it should look like this:

![azure_blob_connector_auth](https://happypathspublic.blob.core.windows.net/assets/batch_scoring_for_dl/azure_blob_connector_auth.PNG)

For both of these connectors, click on the orange bar at the top to authenticate.

Once authenticated, your Logic App should be all set up and ready to trigger the workflow.

### Trigger logic app by adding a new video to the Azure blob container

Re-upload the orangutan video to blob with a new name: `new_video.mp4`

In [ ]:
!azcopy \
    --source orangutan.mp4 \
    --destination https://{get_key(env_path, "STORAGE_ACCOUNT_NAME")}.blob.core.windows.net/{get_key(env_path, "STORAGE_CONTAINER_NAME")}/new_video.mp4 \
    --dest-key {get_key(env_path, "STORAGE_ACCOUNT_KEY")} \
    --resume "."

Check that there are items in the Service Bus queue. It will take some time before we see any items in the queue because Logic App has to detect a new video, kick of ACI (which includes downloading the ACI image), perform the preprocessing, and finally add the preprocessed frames to the queue.

In [ ]:
!az servicebus queue show \
    --name {get_key(env_path, "SB_QUEUE")} \
    --namespace-name {get_key(env_path, "SB_NAMESPACE")} \
    --resource-group {get_key(env_path, "RESOURCE_GROUP")} \
    --query 'countDetails.activeMessageCount'

Check the logs from one of the pods in your AKS cluster:

In [ ]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

---

Continue to the next [notebook](/notebooks/05_clean_up.ipynb).